# Payload

In [1]:
PAYLOAD = {
    "datasetName": "random_10",
    "gaParameters": {
        "maxNumberOfIter":300,
        "populationSize": 100,
        "maximumWildCardCount": 1,
        "orientationWeights": [
            1,
            1,
            0.5
        ],
        "geneticAlgorithm": "simpleGa",
        "mate": "normalizedProbabilityVectorSum",
        "mutate": "flipOnePartAtRandom",
        "select": "tournament",
        "objective": "simple",
        "evaluator": "ga",
        "placingHeuristics": "corner",
        "populationDivisionCounts": {
            "elite": 0.2,
            "average": 0.6,
            "worst": 0.2,
            "children": 0.3,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0.1
        },
        "initialPopulationDivisionCounts": {
            "random": 0.7,
            "greedy": 0.3
        }
    },
    "objectiveParameters": {
        "name": "simple",
        "params": {
            "overlappingPenalizationConstant": 0,
            "outsideOfAllocatedAreaPenalizationConstant": 0
        }
    }
}

URL = "http://localhost:9000/compute/dataset"

HEADERS = {
  'Content-Type': 'application/json'
}

# Utils

In [2]:
import os
import time
import requests
import json
from pathlib import Path
import os
import shutil
from datetime import datetime
import math
import copy

def createDirectory(directory, removeAll=False):
    '''
    Creates a directory if it doesn't exist and removes all files
    from the directory if it already exists and removeAll is True.
    '''
    if os.path.exists(directory) and removeAll:
        shutil.rmtree(directory) # Remove all files in the directory
        print(f'All files in {directory} have been removed.')

    if not os.path.exists(directory):
        os.makedirs(directory) # Create the directory
        
    return directory

def checkforFileBlocking(outDir):
    start_time = time.time()  # Get the current time in seconds
    while True:
        if os.path.exists(os.path.join(outDir, "computation_result.json")):
            elapsed_time = time.time() - start_time  # Calculate the elapsed time in seconds
            print(f"{elapsed_time:.2f} sec")
            time.sleep(5)  # weit for file to be written and closed
            break
        else:
            time.sleep(1)  # Pause for 1 second before checking again
            
def findDataset(name):
    for root, dirs, files in os.walk('/home/slarty/Documents/master-thesis/src/impl/public/datasets'):
        for file in files:
            if file == f'{name}.json' or file == name:
                return os.path.join(root, file)
    return None

def calculateLayoutDiagonalLen(datasetName):
    with open(findDataset(datasetName), 'r') as file:
        layout = json.load(file)['data']['layout']
    return math.sqrt(layout['width']**2 + layout['height']**2)
    

# def setOverlappingConstant(data, dataset):
#     with open(findDataset(dataset), 'r') as file:
#         layout = json.load(file)['data']['layout']
#     data['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
    
def savePayload(payload, directory):
    with open(f'{directory}/payload.json', 'w') as file:
        json.dump(payload, file, indent=4)
    
def sendPayloadBlocking(payload, outDir):
    r = requests.request("POST", URL, headers=HEADERS, data=json.dumps(payload))
    print(r.json())
    src = r.json()['outputDirectory'] 
    savePayload(payload, outDir)
    checkforFileBlocking(src)
    shutil.copytree(src, outDir + '/' + src.split('/')[-1])
    
def createOutDir(name):
    directory = f'out/{name}_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
    createDirectory(directory)
    return directory

def printRun(run, runMax):
    print(f'====== RUN {run}/{runMax} ======')
    
def setCommonAttributes(payload, dataset,size):
    print(f'{dataset}')
    payload['datasetName'] = dataset
    payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateLayoutDiagonalLen(dataset)
    payload['objectiveParameters']['params']['outsideOfAllocatedAreaPenalizationConstant'] = calculateLayoutDiagonalLen(dataset)
    payload['gaParameters']['populationSize'] = size * 50
    payload['gaParameters']['maxNumberOfIter'] = 300

# Hyperparameters

## 1. Max number of iter

In [3]:
# zopakovat nekolikrat pro maxNumberOfIter=500, mozna by slo i vic, treba 800 nebo 1000
outDirBase = createOutDir('max_number_of_iter')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20',20)]:
        setCommonAttributes(payload, dataset, size)
        for maxNumberOfIter in [100, 200, 300, 400, 500]:
            print(f'\tmaxNumberOfIter={maxNumberOfIter}')
            payload['gaParameters']['maxNumberOfIter'] = maxNumberOfIter
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_10
	maxNumberOfIter=100
{'id': 'random_10_BA14E', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/052_random_10_BA14E'}
65.06 sec
	maxNumberOfIter=200
{'id': 'random_10_4B815', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/053_random_10_4B815'}
118.12 sec
	maxNumberOfIter=300
{'id': 'random_10_33507', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/054_random_10_33507'}
178.18 sec
	maxNumberOfIter=400
{'id': 'random_10_690C0', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/055_random_10_690C0'}
237.24 sec
	maxNumberOfIter=500
{'id': 'random_10_7BC04', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/056_random_10_7BC04'}
299.30 sec
random_20
	maxNumberOfIter=100
{'id': 'random_20_38AA7', 'outputDirectory': '/home/slarty/Documents/mast

## 2. Population size

In [4]:
outDirBase = createOutDir('population_size')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20',20)]:
        setCommonAttributes(payload, dataset, size)
        for scalingFactor in [1,3,5,8,10]:
            populationSize = size * scalingFactor
            print(f'\tscalingFactor={scalingFactor}, populationSize={populationSize}')
            payload['gaParameters']['populationSize'] = populationSize
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_10
	scalingFactor=1, populationSize=10
{'id': 'random_10_FB4DB', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/062_random_10_FB4DB'}
6.01 sec
	scalingFactor=3, populationSize=30
{'id': 'random_10_F8470', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/063_random_10_F8470'}
13.01 sec
	scalingFactor=5, populationSize=50
{'id': 'random_10_FEEC4', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/064_random_10_FEEC4'}
19.02 sec
	scalingFactor=8, populationSize=80
{'id': 'random_10_22D81', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/065_random_10_22D81'}
30.03 sec
	scalingFactor=10, populationSize=100
{'id': 'random_10_EA542', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/066_random_10_EA542'}
36.04 sec
random_20
	scalingFactor=1, popu

## 3. Maximum wild card count

In [5]:
outDirBase = createOutDir('maximum_wild_card_count')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10)]:
        setCommonAttributes(payload, dataset, size)
        for maximumWildCardCount in range(10):
            print(f'\tmaximumWildCardCount={maximumWildCardCount}')
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_10
	maximumWildCardCount=0
{'id': 'random_10_61B19', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/072_random_10_61B19'}
101.10 sec
	maximumWildCardCount=1
{'id': 'random_10_B6B4F', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/073_random_10_B6B4F'}
176.18 sec
	maximumWildCardCount=2
{'id': 'random_10_1546D', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/074_random_10_1546D'}
309.31 sec
	maximumWildCardCount=3
{'id': 'random_10_1B246', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/075_random_10_1B246'}
512.51 sec
	maximumWildCardCount=4
{'id': 'random_10_A99F1', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/076_random_10_A99F1'}
762.76 sec
	maximumWildCardCount=5
{'id': 'random_10_E9B55', 'outputDirectory': '/home/slarty/Docum

## 4. Orientation weights

In [6]:
outDirBase = createOutDir('orientation_weights')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for w in [0, 0.2, 0.5, 0.7, 1]:
            orientationWeights = [1,1,w]
            print(f'\torientationWeights={orientationWeights}')
            payload['gaParameters']['orientationWeights'] = orientationWeights
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_10
	orientationWeights=[1, 1, 0]
{'id': 'random_10_0FCBB', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/082_random_10_0FCBB'}
175.17 sec
	orientationWeights=[1, 1, 0.2]
{'id': 'random_10_AE239', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/083_random_10_AE239'}
175.17 sec
	orientationWeights=[1, 1, 0.5]
{'id': 'random_10_FA223', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/084_random_10_FA223'}
175.17 sec
	orientationWeights=[1, 1, 0.7]
{'id': 'random_10_6FD0A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/085_random_10_6FD0A'}
175.17 sec
	orientationWeights=[1, 1, 1]
{'id': 'random_10_D9B32', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/086_random_10_D9B32'}
177.18 sec
random_20
	orientationWeights=[1, 1, 0]
{'id': 'rand

## 5. Population division counts

In [7]:
counts =  [
    # no random, add to children
    {
            "elite": 0.2,
            "average": 0.6,
            "worst": 0.2,
            "children": 0.4,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0
        },
    
        # "populationDivisionCounts": {
        #     "elite": 0.2,
        #     "average": 0.6,
        #     "worst": 0.2,
        #     "children": 0.3,
        #     "mutant": 0.2,
        #     "winner": 0.2,
        #     "random": 0.1
        # },
    # no elite, add to children and equaly to average|worst
    {
            "elite": 0.01,
            "average": 0.69,
            "worst": 0.3,
            "children": 0.49,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0.1
        },
    # no elite and random, add to children, add equaly to average|worst
    {
            "elite": 0.01,
            "average": 0.69,
            "worst": 0.3,
            "children": 0.59,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0
        }
]

outDirBase = createOutDir('initial_population_division_counts')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for populationDivisionCounts in counts:
            print(f'\tpopulationDivisionCounts={populationDivisionCounts}')
            payload['gaParameters']['populationDivisionCounts'] = populationDivisionCounts
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_10
	populationDivisionCounts={'elite': 0.2, 'average': 0.6, 'worst': 0.2, 'children': 0.4, 'mutant': 0.2, 'winner': 0.2, 'random': 0}
{'id': 'random_10_E8E3A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/092_random_10_E8E3A'}
176.18 sec
	populationDivisionCounts={'elite': 0.01, 'average': 0.69, 'worst': 0.3, 'children': 0.49, 'mutant': 0.2, 'winner': 0.2, 'random': 0.1}
{'id': 'random_10_B5344', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/093_random_10_B5344'}
345.34 sec
	populationDivisionCounts={'elite': 0.01, 'average': 0.69, 'worst': 0.3, 'children': 0.59, 'mutant': 0.2, 'winner': 0.2, 'random': 0}
{'id': 'random_10_1EB0E', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/094_random_10_1EB0E'}
344.34 sec
random_20
	populationDivisionCounts={'elite': 0.2, 'average': 0.6, 'worst': 0.2, 'children': 0.4, 'mutant': 0.2

## 6. Initial population division counts

In [8]:
outDirBase = createOutDir('initial_population_division_counts')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for randomPercent in [0, 0.3, 0.5, 0.8, 1]:
            print(f'\trandomPercent={randomPercent}')
            payload['gaParameters']['initialPopulationDivisionCounts']['random'] = randomPercent
            payload['gaParameters']['initialPopulationDivisionCounts']['greedy'] = 1 - randomPercent
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_10
	randomPercent=0
{'id': 'random_10_9BF3F', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/098_random_10_9BF3F'}
177.18 sec
	randomPercent=0.3
{'id': 'random_10_5841D', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/099_random_10_5841D'}
176.18 sec
	randomPercent=0.5
{'id': 'random_10_DBFD8', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/100_random_10_DBFD8'}
176.18 sec
	randomPercent=0.8
{'id': 'random_10_F76A8', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/101_random_10_F76A8'}
178.18 sec
	randomPercent=1
{'id': 'random_10_EE909', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/102_random_10_EE909'}
181.18 sec
random_20
	randomPercent=0
{'id': 'random_20_8D97A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/imp

## 7. Overlapping penalization constant

In [9]:
outDirBase = createOutDir('overlapping_penalization_constant')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for scalingFactor in [0.5, 1, 2, 3, 4]:
            overlappingPenalizationConstant = calculateLayoutDiagonalLen(dataset) * scalingFactor
            print(f'\tscalingFactor={scalingFactor}, overlappingPenalizationConstant={overlappingPenalizationConstant:.2f}')
            payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = overlappingPenalizationConstant
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_10
	scalingFactor=0.5, overlappingPenalizationConstant=15.31
{'id': 'random_10_89986', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/108_random_10_89986'}
176.18 sec
	scalingFactor=1, overlappingPenalizationConstant=30.61
{'id': 'random_10_30C0B', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/109_random_10_30C0B'}
176.18 sec
	scalingFactor=2, overlappingPenalizationConstant=61.22
{'id': 'random_10_6727B', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/110_random_10_6727B'}
177.18 sec
	scalingFactor=3, overlappingPenalizationConstant=91.83
{'id': 'random_10_4CC2E', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/111_random_10_4CC2E'}
177.18 sec
	scalingFactor=4, overlappingPenalizationConstant=122.44
{'id': 'random_10_3703C', 'outputDirectory': '/home/slarty/Documents/master-t

## 8. Outside of allocated area penalization constant

In [10]:
outDirBase = createOutDir('outside_of_allocated_area_penalization_constant')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('random_20', 20)]:
        setCommonAttributes(payload, dataset, size)
        for scalingFactor in [0.5, 1, 2, 3, 4]:
            outsideOfAllocatedAreaPenalizationConstant = calculateLayoutDiagonalLen(dataset) * scalingFactor
            print(f'\tscalingFactor={scalingFactor}, overlappingPenalizationConstant={outsideOfAllocatedAreaPenalizationConstant:.2f}')
            payload['objectiveParameters']['params']['outsideOfAllocatedAreaPenalizationConstant'] = outsideOfAllocatedAreaPenalizationConstant
            sendPayloadBlocking(payload, outDir)
print('\nComputation done')

====== RUN 1/1 ======
random_10
	scalingFactor=0.5, overlappingPenalizationConstant=15.31
{'id': 'random_10_6F000', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/118_random_10_6F000'}
183.18 sec
	scalingFactor=1, overlappingPenalizationConstant=30.61
{'id': 'random_10_D5699', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/119_random_10_D5699'}
182.18 sec
	scalingFactor=2, overlappingPenalizationConstant=61.22
{'id': 'random_10_8269B', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/120_random_10_8269B'}
184.18 sec
	scalingFactor=3, overlappingPenalizationConstant=91.83
{'id': 'random_10_F7CB7', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/121_random_10_F7CB7'}
192.19 sec
	scalingFactor=4, overlappingPenalizationConstant=122.44
{'id': 'random_10_C6A07', 'outputDirectory': '/home/slarty/Documents/master-t

# Measurements

## Population size x best solution

In [15]:
outDirBase = createOutDir('population_size')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('cluster_3_10', 30), ('cluster_4_20',80)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maxNumberOfIter'] = 30
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/5 ======
{'id': 'cluster_3_10_3E78A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/008_cluster_3_10_3E78A'}


KeyboardInterrupt: 

## Overlapping Penalization Constant

In [11]:
outDirBase = createOutDir('overlapping_penalization_constant')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

def getLayoutWidthAndHeight(dataset):
    with open(findDataset(dataset), 'r') as file:
        layout = json.load(file)['data']['layout']
    return layout['width'], layout['height']

def calculateDiagonalLength(dataset):
    w,h = getLayoutWidthAndHeight(dataset)
    return math.sqrt(w**2 + h**2)


def setOverlappingConstant(data, dataset):
    with open(findDataset(dataset), 'r') as file:
        layout = json.load(file)['data']['layout']
    data['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
    
for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('packing_10', 10)]:
        for k in [2, 3, 4, 6]:
            print(f'{dataset}, {k}')
            payload['datasetName'] = dataset
            payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * k
            payload['gaParameters']['populationSize'] = size * 50
            payload['gaParameters']['maxNumberOfIter'] = 300
            payload['gaParameters']['maximumWildCardCount'] = 0
            sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/5 ======
random_10, 2
{'id': 'random_10_D9842', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/094_random_10_D9842'}
113.12 sec
random_10, 3
{'id': 'random_10_73195', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/095_random_10_73195'}
114.11 sec
random_10, 4
{'id': 'random_10_AD7EE', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/096_random_10_AD7EE'}
113.11 sec
random_10, 6
{'id': 'random_10_68D8F', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/097_random_10_68D8F'}
113.11 sec
packing_10, 2
{'id': 'packing_10_1457A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/098_packing_10_1457A'}
94.09 sec
packing_10, 3
{'id': 'packing_10_B0ACF', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/099_packing_10_B0

## Random

In [20]:
outDirBase = createOutDir('random')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_20',20)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
{'id': 'random_20_FDC76', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/148_random_20_FDC76'}
990.00 sec

Computation done


## Cluster

In [22]:
outDirBase = createOutDir('cluster')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

# 10.4 - testing maximum wild card count = 1
for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    # for dataset, size in [('cluster_3_6', 18)]:
    for dataset, size in [('cluster_4_5', 20)]:
        print(f'{dataset}')
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
cluster_4_5
{'id': 'cluster_4_5_606A8', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/150_cluster_4_5_606A8'}
855.86 sec

Computation done


## Packing

In [ ]:
outDirBase = createOutDir('packing')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('packing_10',10), ('packing_20',20)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

## Biased sparse cluster

In [25]:
outDirBase = createOutDir('biased_sparse_cluster')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('biased_sparse_cluster_3_5', 15)]:
        print(f'maximumWildCardCount={maximumWildCardCount}, dataset={dataset}')
        payload['datasetName'] = dataset
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
maximumWildCardCount=2, dataset=biased_sparse_cluster_3_5
{'id': 'biased_sparse_cluster_3_5_F769D', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/152_biased_sparse_cluster_3_5_F769D'}
686.71 sec

Computation done


## Maximum wild card count vs computation time

In [13]:
outDirBase = createOutDir('wild_card_comp_time')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('packing_10',10)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 300
        
        for maximumWildCardCount in range(0,size+1):
            print(maximumWildCardCount)
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount
            sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/5 ======
0
{'id': 'packing_10_9D560', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/139_packing_10_9D560'}
180.18 sec
1
{'id': 'packing_10_4B0EB', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/140_packing_10_4B0EB'}
309.31 sec
2
{'id': 'packing_10_94F18', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/141_packing_10_94F18'}


KeyboardInterrupt: 

## Bruteforce

In [12]:
outDirBase = createOutDir('brute')
payload = copy.deepcopy(PAYLOAD)

data = [('random_10',10), ('random_20',20), ('cluster_3_10',30)]
data = [('cluster_3_6', 18)]

for dataset, size in data:
    print(f'{dataset}')
    setOverlappingConstant(payload, dataset)
    payload['datasetName'] = dataset
    payload['gaParameters']['populationSize'] = 100 * size
    payload['gaParameters']['maxNumberOfIter'] = 1000
    payload['gaParameters']['geneticAlgorithm'] = 'probBrute'
    sendPayloadBlocking(payload, outDirBase)

print('\nComputation done')

random_10
775.77 sec
random_20
3908.93 sec
cluster_3_10
9184.24 sec

Computation done


## London National Gallery

In [32]:
outDirBase = createOutDir('london_nationa_gallery')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('london_gallery_wall',7)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 10
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 5
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
{'id': 'london_gallery_wall_4C686', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/157_london_gallery_wall_4C686'}
6.01 sec

Computation done
